In [1]:
import numpy as np
import pandas as pd
import dataset as ds

from datetime import datetime

In [2]:
def getNumCrowd():
    return ds.dbNumCrowd()

num_rowd = getNumCrowd()
num_rowd.head(5)

,recordtime,in_num,out_num,position,storeid
0,2018-08-03 16:08:00,0,4,F02,29
1,2018-05-03 10:23:00,2,0,Floor 02,28
2,2018-05-03 10:25:00,1,1,Floor 02,28
3,2018-05-03 10:27:00,2,0,Floor 02,28
4,2018-05-03 10:29:00,1,0,Floor 02,28


In [ ]:
# Daily:   [y = None, d = '2023-03-10', w = None                 , m = None   , q = None]
# Weekly:  [y = 2023, d = None        , w = 'WK1 (01/01 - 01/01)', m = None   , q = None]
# Monthly: [y = 2023, d = None        , w = None                 , m = 'April', q = None]
# Quarter: [y = 2023, d = None        , w = None                 , m = None   , q = 0   ]
# Yearly:  [y = 2023, d = None        , w = None                 , m = None   , q = None]

# 2023-04-20 - <class 'datetime.date'>
# WK1 (01/01 - 01/01) - <class 'str'>
# April - <class 'str'>
# 0 - <class 'int'>
# 2023 - <class 'int'>

In [3]:
def getWeekNums(year):
    start_date = '1/1/' + year
    end_date = '12/31/' + year
    data = pd.date_range(start = start_date, end = end_date, freq = 'D')
    data = pd.DataFrame(data, columns = ['date'])
    data['year_calendar'] = data['date'].dt.isocalendar().year
    data['week_calendar'] = data['date'].dt.isocalendar().week

    group = data.groupby(['year_calendar', 'week_calendar']) \
                .agg({'date': ['min', 'max']}).reset_index()

    group['week_num'] = np.where(group['week_calendar'][0] == 52,
                                 group['week_calendar'] + 1, group['week_calendar'])
    if group['week_num'][0] == 53: group.at[0, 'week_num'] = 1

    group['week'] = 'WK' + group['week_num'].astype(str) + \
                    ' (' + group['date']['min'].dt.strftime('%d/%m') + \
                    ' - ' + group['date']['max'].dt.strftime('%d/%m') + ')'

    return group

In [4]:
store = 0

date_selected = datetime.strptime('2022-03-17', '%Y-%m-%d').date()

year_selected = 2023

weeks = getWeekNums(str(year_selected))
week_selected = (weeks.loc(0)[0][0], weeks.loc(0)[0][1])

month_selected = 'January'

quarter_selected = 0

In [9]:
data = num_rowd
# data.recordtime.dt.date
# data.recordtime.dt.year
data.recordtime.dt.year
# data.recordtime.dt.isocalendar().year
# data.recordtime.dt.isocalendar().week
# data.recordtime.dt.strftime('%B')
# data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int)

0          2018
1          2018
2          2018
3          2018
4          2018
           ... 
2986826    2023
2986827    2023
2986828    2023
2986829    2023
2986830    2023
Name: recordtime, Length: 2986831, dtype: int64

In [5]:
def filter_data(data, store, date = None, year = None, week = None, month = None, quarter = None):
    if (not year) and date:
        # print('Daily')
        data = data[data.recordtime.dt.date == date]
    else:
        # print('Yearly')
        data = data[data.recordtime.dt.year == year]
        if week:
            # print('Weekly')
            data = data[(data.recordtime.dt.isocalendar().year == week[0]) & (data.recordtime.dt.isocalendar().week == week[1])]
        elif month:
            # print('Monthly')
            data = data[data.recordtime.dt.strftime('%B') == month]
        elif quarter:
            # print('Quarter')
            data = data[data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int) == quarter + 1]

    if store > 0:
        data = data[data.storeid == store]

    return data.reset_index(drop = True)

In [ ]:
df_daily = filter_data(num_rowd.copy(), store, date = date_selected)
df_daily

In [ ]:
df_weekly = filter_data(num_rowd.copy(), store, year =  year_selected, week = week_selected)
df_weekly

In [ ]:
df_monthly = filter_data(num_rowd.copy(), store, year =  year_selected, month = month_selected)
df_monthly

In [ ]:
df_quarter = filter_data(num_rowd.copy(), store, year =  year_selected, quarter = quarter_selected)
df_quarter

In [6]:
df_yearly = filter_data(num_rowd.copy(), store, year =  year_selected)
df_yearly

,recordtime,in_num,out_num,position,storeid
0,2023-02-15 09:49:00,18,5,Parking,36
1,2023-02-15 09:54:00,17,7,Parking,36
2,2023-02-15 09:59:00,12,5,Parking,36
3,2023-02-15 10:04:00,9,5,Parking,36
4,2023-02-15 10:09:00,15,4,Parking,36
...,...,...,...,...,...
207286,2023-04-17 12:10:00,1,0,Phu,35
207287,2023-04-17 12:29:00,10,4,Starbuck,34
207288,2023-04-17 12:18:00,9,0,Floor 04,31
207289,2023-04-17 12:27:00,20,12,Parking,37
